In [26]:
#!usr/bin/python
import csv
import glob
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

testInputFile = "./testInput.csv"
testTargetFile = "./testTarget.csv"

trainInputFiles = glob.glob("./trainInput[1-9].csv")
trainTargetFiles = glob.glob("./trainTarget[1-9].csv")

lambs = np.linspace(0,4,num=41)
y_values = []

def readInData(file, arrayToPopulate):
    ''' Parameters:
        dataPath: CSV file to extract from
        arrayToPopulate: Array to add values to'''
    with open(file, "r") as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            arrayToPopulate.append(row)

def calculateLoss(trainInput, trainTarget, testInput, testTarget, lamb):
    x_bar = []
    for trainInputRow in trainInput:
        x_bar.append([1] + trainInputRow)

    x_bar = np.array(x_bar).astype('float')  # x_bar is actually x_bar transpose from the slides
    
    A = np.matmul(x_bar.transpose(), x_bar)

    b = np.array([[0, 0, 0]])
    for i in range(len(x_bar)):
        b = b + np.multiply(float(trainTarget[i][0]), x_bar[i])
    b = b.transpose()
    w = np.matmul(np.linalg.inv(np.multiply(lamb, np.identity(3)) + A), b)
    # Test w against the test data: Use euclidean loss function on slide 3
    loss = 0

    for i in range(len(testTarget)):
        difference = (np.matmul(w.transpose(), np.transpose(np.array([1] + testInput[i]).astype('float'))) - float(
            testTarget[i][0]))[0] # Maybe I can sift through this line to see if I really need w.transpose()
        loss = loss + (difference * difference)
    lose = 0.5 * loss
    return loss

def sort(input):
	return input[0]

def regress(lamb):
    testInput = []
    testTarget = []
    trainInput = []
    trainTarget = []

    readInData(testInputFile, testInput)
    readInData(testTargetFile, testTarget)
    for inputFile in trainInputFiles:
        readInData(inputFile, trainInput)
    for targetFile in trainTargetFiles:
        readInData(targetFile, trainTarget)
    return calculateLoss(trainInput, trainTarget, testInput, testTarget, lamb)

def cross():
    lambdaWeightMax = []
    lamb=0.2
    for lamb in lambs:
        
        lambdaAverage = []
        for j in range(10):  # 10-fold validation
            testInput = []
            testTarget = []
            trainInput = []
            trainTarget = []

            readInData(trainInputFiles[j], testInput)  # Read in one train data set as test data
            readInData(trainTargetFiles[j], testTarget)  # Read in one train label set as test label
            for l in range(0, j):
                readInData(trainInputFiles[l], trainInput)
                readInData(trainTargetFiles[l], trainTarget)
            for l in range(j + 1, 10):
                readInData(trainInputFiles[l], trainInput)
                readInData(trainTargetFiles[l], trainTarget)

            loss = calculateLoss(trainInput, trainTarget, testInput, testTarget, lamb)
            lambdaAverage.append(loss)
        lambdaWeightMax.append([np.mean(lambdaAverage), lamb])
        y_values.append(np.mean(lambdaAverage))
    bestLambda = min(lambdaWeightMax, key=sort)[1]
    
    print("The accuracy of the best lambda on test set:")
    print(regress(bestLambda))

cross()
plt.plot(lambs, y_values)

['./trainInput5.csv', './trainInput1.csv', './trainInput9.csv', './trainInput2.csv', './trainInput6.csv', './trainInput3.csv', './trainInput8.csv', './trainInput4.csv', './trainInput7.csv']
['./trainTarget5.csv', './trainTarget3.csv', './trainTarget9.csv', './trainTarget4.csv', './trainTarget8.csv', './trainTarget7.csv', './trainTarget6.csv', './trainTarget2.csv', './trainTarget1.csv']


IndexError: ignored